In [1]:
import torch 
import numpy as np
import tqdm
import matplotlib.pyplot as plt 
import json
import pandas as pd

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
import torch

c:\Users\mosta\anaconda3\envs\SQLtoTEXT\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
import os

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
data_path = os.path.join(project_root, 'text_to_sql_generator', 'data', 'processed', 'text2sql_clean.csv')
cleaned_dataset = pd.read_csv(data_path)

In [4]:
cleaned_dataset.head()

,sql_prompt,sql
0,What is the total volume of timber sold by eac...,"SELECT salesperson_id, name, SUM(volume) as to..."
1,List all the unique equipment types and their ...,"SELECT equipment_type, SUM(maintenance_frequen..."
2,How many marine species are found in the South...,SELECT COUNT(*) FROM marine_species WHERE loca...
3,What is the total trade value and average pric...,"SELECT trader_id, stock, SUM(price * quantity)..."
4,Find the energy efficiency upgrades with the h...,"SELECT type, cost FROM (SELECT type, cost, ROW..."


In [5]:

df = pd.DataFrame(cleaned_dataset)

# Format input/output pairs
PREFIX = "translate to SQL: "
df["input_text"] = PREFIX + df["sql_prompt"]
df["target_text"] = df["sql"]


In [6]:
#  Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])


In [7]:
# Load model and tokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [8]:
# Tokenization function
def preprocess(example):
    inputs = tokenizer(example["input_text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example["target_text"], max_length=256, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

In [9]:
tokenized_dataset = dataset.map(preprocess, batched=False)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-sql-finetuned",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=10,  # or less
    max_steps=5000,       # override if needed
    logging_dir="./logs",
    predict_with_generate=True,
)

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
trainer = Seq2SeqTrainer(
    model=model,              # Your model (T5, BART, etc.)
    args=training_args,       # The training arguments you defined
    train_dataset=tokenized_dataset,  # Your preprocessed training data
    tokenizer=tokenizer,      # Tokenizer for your model
    data_collator=data_collator,  # Handles batching and padding
)

C:\Users\mosta\AppData\Local\Temp\ipykernel_60932\3049639269.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()  # ✅ This is the correct method to call

AttributeError: 'Seq2SeqTrainingArguments' object has no attribute 'train'

In [33]:

trainer.train()  # ✅ This is the correct method to call

Step,Training Loss
500,0.146900
1000,0.165000
1500,0.193000
2000,0.181900
2500,0.180400
3000,0.186900
3500,0.169300
4000,0.167800
4500,0.176900
5000,0.182000


TrainOutput(global_step=5000, training_loss=0.17500954437255858, metrics={'train_runtime': 402.2421, 'train_samples_per_second': 24.861, 'train_steps_per_second': 12.43, 'total_flos': 1353418014720000.0, 'train_loss': 0.17500954437255858, 'epoch': 0.1})

In [41]:
result = inference(
    "translate to SQL: What is the total volume of timber sold by each salesperson?",
    max_length=128,
    num_beams=5,
    early_stopping=True
)
print(result[0]['generated_text'])


SELECT salesperson, SUM(volume) as total_volume FROM salespersons GROUP BY salesperson;
